In [1]:
import numpy as np
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
#from keras.models import load_model
import os
import matplotlib.pyplot as plt
import pickle


Path = "/media/ug-ml/Samsung_T5/training3000/AllDirectionDataSet"

ChunkImage = "ChunkImage_"
ChunkLabel = "ChunkLabel_"
NameNumbers = 0
MaxChunk = 101

rng = np.random.default_rng()

Chunk_Size = 8

ShuffleIndex = np.arange(MaxChunk, dtype = np.int)
rng.shuffle(ShuffleIndex)

RatioTrain = 0.9

TrainNo = int(MaxChunk * RatioTrain)
TrainIndex = ShuffleIndex[0:TrainNo]
ValIndex = ShuffleIndex[TrainNo:-1]

In [2]:

os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

PixelDimension = 36


In [3]:
model = models.Sequential()
model.add(layers.SeparableConv2D(128, (3, 3), activation='relu', data_format='channels_first', input_shape=(PixelDimension, 128, 128)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), data_format='channels_first', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), data_format='channels_first',  activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(512, activation='relu', kernel_regularizer = l2(0.0001)))
model.add(layers.Dense(10, activation='softmax', kernel_regularizer = l2(0.0001)))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(learning_rate = 0.0005), metrics=['acc'])

#model.summary()


EarlyStop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
#Checkpoint = ModelCheckpoint("3DCovBest.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)

In [6]:

Epochs = 100
TrainLoadedChunks = 20
EpochsChunk = int(Epochs * TrainIndex.shape[0] / TrainLoadedChunks)


BatchShuffle = np.arange(Chunk_Size * 10 * TrainLoadedChunks, dtype = int).reshape(TrainLoadedChunks * Chunk_Size * 10)
rng.shuffle(BatchShuffle)

index = 0
for i in range(0, EpochsChunk):
    
    train_images = np.zeros(Chunk_Size * 10 * 36 * TrainLoadedChunks * 128 * 128, dtype = np.float32).reshape(TrainLoadedChunks, Chunk_Size * 10, 36, 128, 128)
    train_lab = np.zeros(Chunk_Size * 10 * TrainLoadedChunks, dtype = np.int).reshape(TrainLoadedChunks, Chunk_Size * 10)
    for j in range(0, TrainLoadedChunks):
        index = i * TrainLoadedChunks + j
        TrainChunk_i = TrainIndex[index % TrainIndex.shape[0]]
        #print("Train Chunk", TrainChunk_i)
        
        
        
        
        train_images[j] = np.load(Path + "/" + ChunkImage + str(TrainChunk_i) + ".npy")
        train_lab[j] = np.load(Path + "/" + ChunkLabel + str(TrainChunk_i) + ".npy")
    
    train_images = train_images.reshape(-1, 36, 128, 128)
    train_images = train_images[BatchShuffle]
    train_lab = train_lab.reshape(-1)
    train_lab = to_categorical(train_lab[BatchShuffle])
    
    ValChunk_i = ValIndex[(i * TrainLoadedChunks) % ValIndex.shape[0]]
    #print("Val Chunk", ValChunk_i)
    
    val_images = np.load(Path + "/" + ChunkImage + str(ValChunk_i) + ".npy")
    val_lab = to_categorical(np.load(Path + "/" + ChunkLabel + str(ValChunk_i) + ".npy"))
    
    
    print("EPOCH: ", int(i / TrainIndex.shape[0]), "CHUNK: ", i % TrainIndex.shape[0])
    history = model.fit(train_images, train_lab, epochs=1, batch_size = 64, validation_data = (val_images, val_lab), shuffle = True, callbacks=[EarlyStop]) #, Checkpoint

    #model.save('1000AllDirection.h5')




KeyboardInterrupt: 